<a href="https://colab.research.google.com/github/i2mmmmm/kaggle_flood/blob/main/colab/4.%20ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_USERNAME']='mmmmmlee'
os.environ['KAGGLE_KEY']='b4527f98e27813c6dae3fb199096dcb5'

In [ ]:
!kaggle competitions download -c Playground-series-s4e5

 61% 17.0M/28.0M [00:00<00:00, 64.4MB/s]
100% 28.0M/28.0M [00:00<00:00, 92.5MB/s]


In [ ]:
!unzip 'Playground-series-s4e5.zip'

Archive:  Playground-series-s4e5.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import VotingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import BayesianRidge, SGDRegressor, Ridge
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline
import xgboost as xgb
import lightgbm as lgb

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
# train 표준화, 스케일링
scaler = StandardScaler()
X = train.drop(columns=['FloodProbability', 'id'])
y = train['FloodProbability']
X_scaled = scaler.fit_transform(X)

In [ ]:
# 선형회귀_baseline

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=35)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('R2 Score:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

R2 Score: 0.8449835489068076
MSE: 0.00040395088979260915


In [ ]:
# 다시 feature engineering
train = pd.read_csv('train.csv')

def simplified_getFeats(df):
    num_cols = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement', 'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality', 'Siltation', 'AgriculturalPractices', 'Encroachments', 'IneffectiveDisasterPreparedness', 'DrainageSystems', 'CoastalVulnerability', 'Landslides', 'Watersheds', 'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss', 'InadequatePlanning', 'PoliticalFactors']

    scaler = StandardScaler().fit(df[num_cols])
    df[num_cols] = scaler.transform(df[num_cols])  # Scale early

    # Basic Stats
    df['mean'] = df[num_cols].mean(axis=1)
    df['std'] = df[num_cols].std(axis=1)
    df['max'] = df[num_cols].max(axis=1)
    df['min'] = df[num_cols].min(axis=1)

    # Interaction Features (Simplified)
    df['Climate_Risk'] = df['MonsoonIntensity'] * df['ClimateChange']
    df['Infrastructure_Risk'] = df['DamsQuality'] * df['DrainageSystems']
    df['wet_Risk'] = df['WetlandLoss'] * df['Encroachments']

    return df

train_simp = simplified_getFeats(train)

X = train_simp.drop(columns=['FloodProbability', 'id'])
y = train_simp['FloodProbability']

In [ ]:
# LightGBM 모델 학습

lgbm_params = {
    'num_leaves': 183,
    'learning_rate': 0.01183688880802108,
    'n_estimators': 577,
    'subsample_for_bin': 165697,
    'min_child_samples': 114,
    'reg_alpha': 2.075080888948164e-06,
    'reg_lambda': 3.838938366471552e-07,
    'colsample_bytree': 0.9634044234652241,
    'subsample': 0.9592138618622019,
    'max_depth': 9
}

model_lgb = lgb.LGBMRegressor(**lgbm_params)
model_lgb.fit(X_train, y_train)
y_pred_lgb = model_lgb.predict(X_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.487907 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1843
[LightGBM] [Info] Number of data points in the train set: 1117957, number of used features: 27
[LightGBM] [Info] Start training from score 0.504480
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [ ]:
# 출력
test_simp['FloodProbability'] = y_pred_lgb
test_simp[['id', 'FloodProbability']].to_csv('submission_engi_lgb_para.csv', index=False)

In [ ]:
sub1 = pd.read_csv('submission (1).csv')
sub2 = pd.read_csv('submission (2).csv')
sub3 = pd.read_csv('submission (3).csv')

print(sub1.head())
print(sub2.head())
print(sub3.head())

        id  FloodProbability
0  1117957          0.578705
1  1117958          0.454723
2  1117959          0.448682
3  1117960          0.467035
4  1117961          0.465906
        id  FloodProbability
0  1117957          0.577933
1  1117958          0.456926
2  1117959          0.447441
3  1117960          0.464352
4  1117961          0.466865
        id  FloodProbability
0  1117957          0.578782
1  1117958          0.454955
2  1117959          0.448574
3  1117960          0.466518
4  1117961          0.466017


In [ ]:
# 예측값 평균 내기
ensemble_sub = sub1.copy()
ensemble_sub['FloodProbability'] = (sub1['FloodProbability'] + sub2['FloodProbability'] + sub3['FloodProbability']) / 3

# 앙상블 결과 저장
ensemble_sub.to_csv('submission_ensemble.csv', index=False)

In [ ]:
ensemble2 = sub2.copy()
ensemble2['FloodProbability'] = sub1['FloodProbability']  +sub1['FloodProbability'] - ensemble_sub['FloodProbability']

In [ ]:
ensemble2.to_csv('submission_ensemble2.csv', index=False)

In [ ]:
print(ensemble2.head())

        id  FloodProbability
0  1117957          0.578473
1  1117958          0.455535
2  1117959          0.448232
3  1117960          0.465968
4  1117961          0.466263
